In [2]:
from numpy import loadtxt
import pandas as pd
from sklearn.inspection import permutation_importance
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier,RadiusNeighborsClassifier
import numpy as np
from sklearn.metrics import *
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.svm import LinearSVC,SVC,NuSVC
from sklearn.linear_model import ElasticNetCV, LassoCV, LogisticRegressionCV,RidgeClassifierCV, ElasticNet, Lasso, LogisticRegression, RidgeClassifier, SGDClassifier, PassiveAggressiveClassifier, Perceptron
from sklearn.naive_bayes import BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import csv
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RepeatedStratifiedKFold
import pickle
import ast
from sklearn.calibration import CalibratedClassifierCV

In [3]:
random_num = 1
data = "./归一化后的数据.xlsx"
data_test = "./归一化后的数据.xlsx"

In [19]:
#模型选择
print('[1]ensemble\n[2]tree\n[3]linear\n[4]svm\n[5]bayes\n[6]logistic\n[7]neural network\n[8]neighbor')
leixing = input('请选择类型：')
if leixing == '1':
    print('[a]AdaBoost\n[b]RandomForest\n[c]GradientBoosting\n[d]ExtraTree\n[e]XGBoost')
    model_all = {'a':'AdaBoostClassifier(algorithm="SAMME", learning_rate=0.14999999999999999, random_state=1)',
                 'b':'RandomForestClassifier(criterion= "entropy", max_depth=9, max_features=0.2, n_estimators=97,random_state=1)',
                 'c':'GradientBoostingClassifier(learning_rate=0.09, max_depth=4, max_features=0.06999999999999999, n_estimators=90, subsample=0.7999999999999999, random_state=1)',
                 'd':'ExtraTreesClassifier(criterion="gini", max_depth=12, max_features=0.06999999999999999,n_estimators=64, random_state=1)'}
                 #'e':'XGBClassifier(max_depth=1,random_state=1,seed = 1)'}
elif leixing == '2':
    print('[a]DecisionTree\n[b]ExtraTree')
    model_all = {'a':'DecisionTreeClassifier(criterion="entropy", splitter="best", max_depth=4, max_features=3,random_state=1)', 
                 'b':'ExtraTreeClassifier(criterion="entropy", max_depth=4, max_features=3,splitter="best",random_state=1)'}
elif leixing == '3':
    print('[a]ElasticNet\n[b]LASSO\n[c]Ridge\n[d]SDG\n[e]感知机\n[f]PAC')
    model_all = {#'a': 'ElasticNetCV(random_state = random_num)', 
                 #'b': 'LassoCV(random_state = random_num)',
                'c':'RidgeClassifierCV(alphas=38.1, fit_intercept=True)',#这个模型不能设置随机数种子
                'd':'SGDClassifier(alpha=0.060000000000000005, fit_intercept=True, l1_ratio=0.4, learning_rate="optimal", loss="modified_huber", penalty="elasticnet", random_state=1)',
                'e':'Perceptron(alpha=5.099999999999999e-05, fit_intercept=True, penalty="l2",random_state=1)',
                'f':'PassiveAggressiveClassifier(C=0.007, fit_intercept=True, random_state = 1)'}
elif leixing == '4':
    print('[a]LinearSVC\n[b]SVC\n[c]LDA')
    model_all = {'a': 'LinearSVC(C=0.15000000000000002, dual=False, penalty="l1", random_state=1)',
                 'b': 'SVC(C=113, gamma=0.0009000000000000001, kernel="rbf", random_state=1)'}
                 #'c':'LinearDiscriminantAnalysis(shrinkage="auto", solver="lsqr")'}
elif leixing == '5':
    print('[a]BernoulliNB')
    model_all = {'a':'BernoulliNB(alpha=1.0101010101010102, binarize=0.30303030303030304, class_prior=None, fit_prior=True)'}#这个模型不能设置随机数种子
    
model_want = input('请选择模型:')
model_out = model_all.get(model_want)
print("model_out:",model_out)

# 载入数据集
excelFile = data
dataset = pd.DataFrame(pd.read_excel(excelFile,sheet_name="训练集"))
x_train = dataset[dataset.columns[1:]]
y_train = dataset[['Group']]
excelFile_test = data_test
dataset_test_1 = pd.DataFrame(pd.read_excel(excelFile_test, sheet_name="留出集"))
x_test_1 = dataset_test_1[dataset_test_1.columns[1:]]
y_test_1 = dataset_test_1[['Group']]
dataset_test_2 = pd.DataFrame(pd.read_excel(excelFile_test, sheet_name="西溪验证集"))
x_test_2 = dataset_test_2[dataset_test_2.columns[1:]]
y_test_2 = dataset_test_2[['Group']]
dataset_test_3 = pd.DataFrame(pd.read_excel(excelFile_test, sheet_name="浙一验证集"))
x_test_3 = dataset_test_3[dataset_test_3.columns[1:]]
y_test_3 = dataset_test_3[['Group']]


model =eval(model_out)
# model = CalibratedClassifierCV(base_estimator=model, method='sigmoid', cv=5)
# cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=1)
model.fit(x_train, y_train.values.ravel())
# scores = cross_val_score(model, x_train, y_train.values.ravel(), cv=cv, n_jobs=-1, scoring='roc_auc', verbose=2)
scores = cross_val_score(model, x_train, y_train, cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=10), n_jobs=-1, scoring='roc_auc', verbose=2)
print('train_AUC:', scores.mean())


if leixing == '3' or leixing == '4':
    y_pred = model.decision_function(x_train)
    y_pred_1 = model.decision_function(x_test_1)
    y_pred_2 = model.decision_function(x_test_2)
    y_pred_3 = model.decision_function(x_test_3)
else:    
    y_pred = model.predict_proba(x_train)[:, 1]
    y_pred_1 = model.predict_proba(x_test_1)[:, 1]
    y_pred_2 = model.predict_proba(x_test_2)[:, 1]
    y_pred_3 = model.predict_proba(x_test_3)[:, 1]


auc_1 = roc_auc_score(y_train, y_pred)
auc_2 = roc_auc_score(y_test_1, y_pred_1)
auc_3 = roc_auc_score(y_test_2, y_pred_2)
auc_4 = roc_auc_score(y_test_3, y_pred_3)
print('External validation 训练集AUC:', auc_1)
print('External validation 留出集AUC:', auc_2)
print('External validation 西溪验证集AUC:', auc_3)
print('External validation 浙一验证集AUC:', auc_4)

[1]ensemble
[2]tree
[3]linear
[4]svm
[5]bayes
[6]logistic
[7]neural network
[8]neighbor
请选择类型：5
[a]BernoulliNB
请选择模型:a
model_out: BernoulliNB(alpha=1.0101010101010102, binarize=0.30303030303030304, class_prior=None, fit_prior=True)
train_AUC: 0.9259830147703902
External validation 训练集AUC: 0.9377892488430046
External validation 留出集AUC: 0.9550000000000001
External validation 西溪验证集AUC: 0.8233333333333334
External validation 浙一验证集AUC: 0.8771121351766512


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  45 out of  50 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.0s finished


# y_pred

In [32]:
random_num = 1
data = "./归一化后的数据/归一化后的数据.xlsx"
data_test = "./归一化后的数据/归一化后的数据.xlsx"

#模型选择
print('[1]ensemble\n[2]tree\n[3]linear\n[4]svm\n[5]bayes\n[6]logistic\n[7]neural network\n[8]neighbor')
leixing = input('请选择类型：')
if leixing == '1':
    print('[a]AdaBoost\n[b]RandomForest\n[c]GradientBoosting\n[d]ExtraTree\n[e]XGBoost')
    model_all = {'a':'AdaBoostClassifier(algorithm="SAMME", learning_rate=0.14999999999999999, random_state=1)',
                 'b':'RandomForestClassifier(criterion= "entropy", max_depth=9, max_features=0.2, n_estimators=97,random_state=1)',
                 'c':'GradientBoostingClassifier(criterion="friedman_mse", learning_rate=0.08, loss="exponential", max_depth=6,max_features=0.1, n_estimators=85, subsample=0.6, random_state=1)',
                 'd':'ExtraTreesClassifier(criterion="gini", max_depth=12, max_features=0.06999999999999999,n_estimators=64, random_state=1)'}
                 #'e':'XGBClassifier(max_depth=1,random_state=1,seed = 1)'}
elif leixing == '2':
    print('[a]DecisionTree\n[b]ExtraTree')
    model_all = {'a':'DecisionTreeClassifier(criterion="entropy", splitter="best", max_depth=4, max_features=3,random_state=1)', 
                 'b':'ExtraTreeClassifier(criterion="entropy", max_depth=4, max_features=3,splitter="best",random_state=1)'}
elif leixing == '3':
    print('[a]ElasticNet\n[b]LASSO\n[c]Ridge\n[d]SDG\n[e]感知机\n[f]PAC')
    model_all = {#'a': 'ElasticNetCV(random_state = random_num)', 
                 #'b': 'LassoCV(random_state = random_num)',
                'c':'RidgeClassifierCV(alphas=38.1, fit_intercept=True)',#这个模型不能设置随机数种子
                'd':'SGDClassifier(alpha=0.060000000000000005, fit_intercept=True, l1_ratio=0.4, learning_rate="optimal", loss="modified_huber", penalty="elasticnet", random_state=1)',
                'e':'Perceptron(alpha=5.099999999999999e-05, fit_intercept=True, penalty="l2",random_state=1)',
                'f':'PassiveAggressiveClassifier(C=0.007, fit_intercept=True, random_state = 1)'}
elif leixing == '4':
    print('[a]LinearSVC\n[b]SVC\n[c]LDA')
    model_all = {'a': 'LinearSVC(C=0.15000000000000002, dual=False, penalty="l1", random_state=1)',
                 'b': 'SVC(C=113, gamma=0.0009000000000000001, kernel="rbf", random_state=1)'}
                 #'c':'LinearDiscriminantAnalysis(shrinkage="auto", solver="lsqr")'}
elif leixing == '5':
    print('[a]BernoulliNB')
    model_all = {'a':'BernoulliNB(alpha=1.0101010101010102, binarize=0.30303030303030304, class_prior=None, fit_prior=True)'}#这个模型不能设置随机数种子
    
model_want = input('请选择模型:')
model_out = model_all.get(model_want)
print("model_out:",model_out)
# 载入数据集

excelFile = data
dataset = pd.DataFrame(pd.read_excel(excelFile, sheet_name=""))
x_train = dataset[dataset.columns[1:-1]]
y_train = dataset[['Group']]
excelFile_test = data_test
dataset_test = pd.DataFrame(pd.read_excel(excelFile_test, sheet_name=""))
x_test = dataset_test[dataset_test.columns[1:-1]]
y_test = dataset_test[['Group']]

#模型构建
model =eval(model_out)
model.fit(x_train, y_train.astype('int'))

# AUC
scores = cross_val_score(model, x_train, y_train, cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=10), n_jobs=-1, scoring='roc_auc', verbose=2)#cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=10)5折交叉验证默认跑5次（默认为1），这时候改成10，这个意思是数据集划分重新划分10次，抽出一份作为验证集的过程有50种可能
# 当verbose=0时，表示不输出任何信息；
# 当verbose=1时，表示输出部分信息，例如每次交叉验证的时间；
# 当verbose=2时，表示输出详细信息，例如每个fold的分数等。
print('train_AUC:', scores.mean())

# 保存模型参数到磁盘上
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)
    
# 重新加载模型参数    
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

# 在验证集上评估模型性能

# if leixing == '3' or leixing == '4':#and model_want == 'a'
#     y_pred = model.predict(x_test)
# else:
y_pred = model.predict_proba(x_test)[:, 1]
auc = roc_auc_score(y_test, y_pred)
print('External validation AUC:', auc)

[1]ensemble
[2]tree
[3]linear
[4]svm
[5]bayes
[6]logistic
[7]neural network
[8]neighbor


请选择类型： 2


[a]DecisionTree
[b]ExtraTree


请选择模型: 1


model_out: None


TypeError: eval() arg 1 must be a string, bytes or code object